<a href="https://colab.research.google.com/github/projectCHOI/S-1_volume/blob/master/2%EB%8B%A8%EA%B3%84_231101_LDA%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##LDA

####불러오기

In [ ]:
import warnings
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas==1.5.3

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/3조-에스원 자료/data/total_data_.csv')

In [ ]:
df.head()

,Unnamed: 0,date,media,url,category,category_sub,title,content,URL
0,0,2021-08-13T10:07:00,서울경제,https://news.naver.com/main/read.naver?mode=LS...,경제,증권,"㈜인콘, 공공조달플랫폼 서비스 운영···다양한 서비스 제공","최근 자영업자, 전업주부, N잡러들 사이에서 공공기관을 상대로 진행하는 '공공 조달...",NaN
1,1,2021-08-23T15:17:00,경향신문,https://news.naver.com/main/read.naver?mode=LS...,경제,산업/재계,"인콘, ‘공공조달플랫폼 서비스’ 통해 자영업자 수익창출 도모",주식회사 인콘은 자영업자 및 N잡러를 위해 공공조달 재테크에 대한 준비를 원큐에 끝...,NaN
2,2,2021-12-24T16:25:00,아시아경제,https://news.naver.com/main/read.naver?mode=LS...,경제,증권,"인콘, 도중식·최영훈 각자대표 체제로 변경","인콘은 경영 효율성 제고를 위한 조직 개편 차원에서 도중식, 최영훈 각자대표 체제로...",NaN
3,3,2021-08-31T10:32:00,데일리안,https://news.naver.com/main/read.naver?mode=LS...,경제,경제 일반,"인콘 조달플랫폼, 조달테크 서비스 통해 N잡 시대 새로운 대안으로 떠올라",최근 MZ세대에서는 본업 외 시간을 활용하는 부업을 떠나서 'N잡러'라는 키워드가 ...,NaN
4,4,2021-09-01T09:45:00,매일신문,https://news.naver.com/main/read.naver?mode=LS...,경제,경제 일반,"인콘, MZ세대 ‘N잡러’를 위한 공공조달플랫폼 서비스 제공",'N잡러'를 위한 공공조달플랫폼 서비스가 많은 관심을 받고 있다. 흔히 말하는 MZ...,NaN


##전처리 토큰화
- 형태소 분석 > 정규표현식 > 한글자 제거 > 불용어 처리
- [토큰화 + 토큰리스트] 생성

###형태소 분석

In [ ]:
#형태소 분석기를 사용하자.
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 35.1 MB/s eta 0:00:00


In [ ]:
#형태소 분석기 Mecab
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash ./install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-11-02 11:24:58--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attachment%3B%20fi

In [ ]:
#형태소 분석기 Mecab
from konlpy.tag import Mecab
tokenizer = Mecab()

# "POS" 열을 생성하고 형태소 분석 결과 저장
for i, row in df.iterrows():
    content = row['content']
    if isinstance(content, str):
        nouns = ' '.join(tokenizer.nouns(content))
        df.at[i, 'POS'] = nouns

# 데이터프레임의 구조 확인
print(df[['content', 'POS']].head())

                                             content  \
0  최근 자영업자, 전업주부, N잡러들 사이에서 공공기관을 상대로 진행하는 '공공 조달...   
1  주식회사 인콘은 자영업자 및 N잡러를 위해 공공조달 재테크에 대한 준비를 원큐에 끝...   
2  인콘은 경영 효율성 제고를 위한 조직 개편 차원에서 도중식, 최영훈 각자대표 체제로...   
3  최근 MZ세대에서는 본업 외 시간을 활용하는 부업을 떠나서 'N잡러'라는 키워드가 ...   
4  'N잡러'를 위한 공공조달플랫폼 서비스가 많은 관심을 받고 있다. 흔히 말하는 MZ...   

                                                 POS  
0  최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...  
1  주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...  
2      인 콘 경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 일 공시  
3  최근 세대 본업 외 시간 활용 부업 키워드 주목 이 종사 본업 외 파트타임 바 음식...  
4  공공 조달 플랫 폼 서비스 관심 말 세대 본업 외 시간 활용 부업 이젠 선택 필수 ...  


In [ ]:
#형태소 분석기 Okt

# from konlpy.tag import Okt

# # Okt 형태소 분석기 초기화
# tokenizer = Okt()

# # "POS" 열을 생성하고 형태소 분석 결과 저장
# for i, row in df.iterrows():
#     content = row['content']
#     if isinstance(content, str):
#         pos_result = ' '.join(tokenizer.morphs(content))
#         df.at[i, 'POS'] = pos_result

# # 데이터프레임의 구조 확인
# print(df[['content', 'POS']].head())

In [ ]:
df.head(2)

,Unnamed: 0,date,media,url,category,category_sub,title,content,URL,POS
0,0,2021-08-13T10:07:00,서울경제,https://news.naver.com/main/read.naver?mode=LS...,경제,증권,"㈜인콘, 공공조달플랫폼 서비스 운영···다양한 서비스 제공","최근 자영업자, 전업주부, N잡러들 사이에서 공공기관을 상대로 진행하는 '공공 조달...",NaN,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...
1,1,2021-08-23T15:17:00,경향신문,https://news.naver.com/main/read.naver?mode=LS...,경제,산업/재계,"인콘, ‘공공조달플랫폼 서비스’ 통해 자영업자 수익창출 도모",주식회사 인콘은 자영업자 및 N잡러를 위해 공공조달 재테크에 대한 준비를 원큐에 끝...,NaN,주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...


###정규표현식

In [ ]:
import re

# 정규 표현식 패턴
regex_pattern = r'\<[^\>]*\>|\&#8203;``【oaicite:0】``&#8203;]*\】|\[[^\)]*\]|\([^\)]*\)|[0-9]*\.[0-9]*?\.[0-9]*|[a-zA-Z]*@[a-zA-Z]*\.[a-zA-Z]*\.?[a-zA-Z]*|Copyright|ⓒ|스포츠서울&sportsseoul.com|제공'

# "RE" 열을 생성하고 정규 표현식을 적용하여 분류된 결과 저장
df['RE'] = df['POS'].apply(lambda text: re.sub(regex_pattern, ' ', text))

# 데이터프레임의 구조 확인
print(df[['POS', 'RE']].head())

                                                 POS  \
0  최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...   
1  주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...   
2      인 콘 경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 일 공시   
3  최근 세대 본업 외 시간 활용 부업 키워드 주목 이 종사 본업 외 파트타임 바 음식...   
4  공공 조달 플랫 폼 서비스 관심 말 세대 본업 외 시간 활용 부업 이젠 선택 필수 ...   

                                                  RE  
0  최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...  
1  주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...  
2      인 콘 경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 일 공시  
3  최근 세대 본업 외 시간 활용 부업 키워드 주목 이 종사 본업 외 파트타임 바 음식...  
4  공공 조달 플랫 폼 서비스 관심 말 세대 본업 외 시간 활용 부업 이젠 선택 필수 ...  


In [ ]:
df.head(2)

,Unnamed: 0,date,media,url,category,category_sub,title,content,URL,POS,RE
0,0,2021-08-13T10:07:00,서울경제,https://news.naver.com/main/read.naver?mode=LS...,경제,증권,"㈜인콘, 공공조달플랫폼 서비스 운영···다양한 서비스 제공","최근 자영업자, 전업주부, N잡러들 사이에서 공공기관을 상대로 진행하는 '공공 조달...",NaN,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...
1,1,2021-08-23T15:17:00,경향신문,https://news.naver.com/main/read.naver?mode=LS...,경제,산업/재계,"인콘, ‘공공조달플랫폼 서비스’ 통해 자영업자 수익창출 도모",주식회사 인콘은 자영업자 및 N잡러를 위해 공공조달 재테크에 대한 준비를 원큐에 끝...,NaN,주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...,주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...


###한글자 제거

In [ ]:
# "RE-1" 열을 생성하고 글자 수 1 이하인 데이터를 제거하여 저장
df['RE-1'] = df['RE'].apply(lambda text: ' '.join(word for word in text.split() if len(word) > 1))

# 데이터프레임의 구조 확인
print(df[['RE', 'RE-1']].head())

                                                  RE  \
0  최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...   
1  주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...   
2      인 콘 경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 일 공시   
3  최근 세대 본업 외 시간 활용 부업 키워드 주목 이 종사 본업 외 파트타임 바 음식...   
4  공공 조달 플랫 폼 서비스 관심 말 세대 본업 외 시간 활용 부업 이젠 선택 필수 ...   

                                                RE-1  
0  최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...  
1  주식회사 자영업 공공 조달 재테크 준비 공공 조달 플랫 해당 서비스 공공 조달 물품...  
2            경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 공시  
3  최근 세대 본업 시간 활용 부업 키워드 주목 종사 본업 파트타임 음식 배달 서비스 ...  
4  공공 조달 플랫 서비스 관심 세대 본업 시간 활용 부업 이젠 선택 필수 추세 파트타...  


In [ ]:
df.head(2)

,Unnamed: 0,date,media,url,category,category_sub,title,content,URL,POS,RE,RE-1
0,0,2021-08-13T10:07:00,서울경제,https://news.naver.com/main/read.naver?mode=LS...,경제,증권,"㈜인콘, 공공조달플랫폼 서비스 운영···다양한 서비스 제공","최근 자영업자, 전업주부, N잡러들 사이에서 공공기관을 상대로 진행하는 '공공 조달...",NaN,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...
1,1,2021-08-23T15:17:00,경향신문,https://news.naver.com/main/read.naver?mode=LS...,경제,산업/재계,"인콘, ‘공공조달플랫폼 서비스’ 통해 자영업자 수익창출 도모",주식회사 인콘은 자영업자 및 N잡러를 위해 공공조달 재테크에 대한 준비를 원큐에 끝...,NaN,주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...,주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...,주식회사 자영업 공공 조달 재테크 준비 공공 조달 플랫 해당 서비스 공공 조달 물품...


###불용어 처리

In [ ]:
stop_pos = ['Noun', 'Josa', 'Alpha', 'Punctuation', 'Suffix']
stop_word = ['은', '는', '이', '가', '을', '를', '에', '에서', '로', '으로',
'와', '과', '의', '처럼', '1분기', '2분기', '3분기', '4분기', '경제', '분석',
'경기', '상승', '하강', '급등', '급락', '회복', '상장', '공모주', '우승', '태권도',
'금매달', '시장', '기업', '텔레콤', '사업', '가치', '예측', '금융', '은행', '뱅크',
'올해', '증권', '현대', '평가', '지분', '성장', '공격', '랜섬', '웨어', '스퀘어',
'버스', '정보', '스가', '양사', '협약', '입니다', '체계', '차량', '미래', '너스',
'구주', '파트', '박정호', '대회', '철회', '시즌', '골프', '선수', '교육', '여자',
'분기', '매출', '대표', '유해', '스퀘어', '주주', '최종', '쉴더', '영업', '일감',
'사이버', '에너지', '너지', '에너', '이익', '채용', '서구', '발인', '호실', '남편',
'별세', '란스', '세브', '란스', '광주광역시', '병원', '신촌', '부친상', '장인상', '해양',
'부상', '관내', '광주', '조선', '구청장', '대우', '측정기', '용인', '신임', '내청', '이사',
'총회', '주총', '이사', '임원', '는데요', '김철수', '마무리', '경영', '경제', '은', '는',
'이', '가', '을', '를', '에', '에서', '로', '으로', '와', '과', '의', '처럼', '같이',
'만', '까지', '부터', '에게', '한테', '에서', '에서부터', '로부터', '에게로', '에서도',
'에도', '하고', '이나', '나', '라고', '이라고', '며', '라며', '에게는', '에겐', '에',
'께', '랑', '하다', '라서', '에서도', '에도', '라면', '만큼', '마냥', '에는', '엔', '에서만',
'로만', '처럼', '같이', '만으로', '부터', '으로서', '로서', '대로', '에게는', '한테도', '뿐',
'야', '이야', '이요', '이라요', '하죠', '하죠', '으로써', '로써', '이란', '는데', '더라도',
'다가', '이에요', '이면', '이었', '더', '죠', '고', '요', '에다', '에다가', '마저', '마저도',
'잘', '빠르게', '조용히', '무척', '너무', '매우', '아주', '정말', '과연', '전혀', '더', '더욱',
'가장', '반드시', '아직', '그래도', '비로소', '결국', '마침내', '아무튼', '약간', '대략', '대체로',
'이미', '좀', '몇몇', '모두', '갑자기', '뜻밖에', '한동안', '한두 번', '언제나', '이미', '최근',
'또', '바로', '뒤', '앞', '거의', '대개', '끝내', '시작하여', '뒤이어', '따라', '함께', '같이'
'프로','스토어','투자','증시','청약','스토','토어','선두','원스','메타',
'버스','타버','대어','매각','케이','코스피','코스닥','브로드밴드','심사','희망',
'기관','투자','그룹','심사','대상','기록','예비','기사','추진','자금',
'공동','부진','확정','반도체','회사','담당','수익','내년','협력','마지막',
'지원','금리','이번','올해','지난해','계획','이날','진행','공개','업체',
'과정','물리','마켓','합병','증가','투자자','업계','관리','국내','전년',
'외국인','이디스','호컴','맥스','주가','비즈','니스','씨티','비즈]','순매도',
'테마','연관','주일','내역','나무','매매','전일','작성','메인','테라',
'매수','등락','하이트','파인','플러스','효성','전자','홀딩스','에스','자체',
'라운드','수상','순위','협의','가구','연속','출발','스와','랭킹','승부',
'주인공','배임','강원','유입','도전','부정','현지','중앙','조성','응원',
'점수','눈높이','횡령','부장','부족','블록체인','엠투엠','플러스','스원','필로',
'공모','사장','모가','모빌리티','선임','출전','장애','텔레','배당','거래소',
'주관','프로그램','취업','성적','스피커','한네트','시스','거래','기간','시간',
'테크','개발','기자','배당','환원','소각','증권사','매입','배당금','자산',
'솔루션','오전','의혹','본사','하청','격차','장지호','거래','시설','물량',
'투어','시가총액','주행','배송','생각','합계','컬리','부회장','은퇴','제주',
'대비','분위기','커머스','단위','오픈','한국','차지','직무','사회','카드',]  # 이어지는 불용어 리스트

# "PRO" 열을 생성하고 불용어 처리된 결과 저장
def preprocess(text):
  text = str(text).split()
  text = [i for i in text if len(i) > 1]
  text = [i for i in text if i not in stop_pos]
  text = [i for i in text if i not in stop_word]
  return text

df['PRO'] = df['RE-1'].apply(preprocess)

# 데이터프레임의 구조 확인
print(df[['RE-1', 'PRO']].head())


                                                RE-1  \
0  최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...   
1  주식회사 자영업 공공 조달 재테크 준비 공공 조달 플랫 해당 서비스 공공 조달 물품...   
2            경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 공시   
3  최근 세대 본업 시간 활용 부업 키워드 주목 종사 본업 파트타임 음식 배달 서비스 ...   
4  공공 조달 플랫 서비스 관심 세대 본업 시간 활용 부업 이젠 선택 필수 추세 파트타...   

                                                 PRO  
0  [영업자, 전업주부, 사이, 공공, 상대, 공공, 조달, 재테크, 화제, 공공, 조...  
1  [주식회사, 자영업, 공공, 조달, 재테크, 준비, 공공, 조달, 플랫, 해당, 서...  
2      [효율, 제고, 조직, 개편, 차원, 중식, 최영훈, 각자, 체제, 변경, 공시]  
3  [세대, 본업, 활용, 부업, 키워드, 주목, 종사, 본업, 파트타임, 음식, 배달...  
4  [공공, 조달, 플랫, 서비스, 관심, 세대, 본업, 활용, 부업, 이젠, 선택, ...  


In [ ]:
df.head()

,Unnamed: 0,date,media,url,category,category_sub,title,content,URL,POS,RE,RE-1,PRO
0,0,2021-08-13T10:07:00,서울경제,https://news.naver.com/main/read.naver?mode=LS...,경제,증권,"㈜인콘, 공공조달플랫폼 서비스 운영···다양한 서비스 제공","최근 자영업자, 전업주부, N잡러들 사이에서 공공기관을 상대로 진행하는 '공공 조달...",NaN,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...,최근 영업자 전업주부 사이 공공 기관 상대 진행 공공 조달 재테크 화제 공공 조달 ...,"[영업자, 전업주부, 사이, 공공, 상대, 공공, 조달, 재테크, 화제, 공공, 조..."
1,1,2021-08-23T15:17:00,경향신문,https://news.naver.com/main/read.naver?mode=LS...,경제,산업/재계,"인콘, ‘공공조달플랫폼 서비스’ 통해 자영업자 수익창출 도모",주식회사 인콘은 자영업자 및 N잡러를 위해 공공조달 재테크에 대한 준비를 원큐에 끝...,NaN,주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...,주식회사 콘 자영업 공공 조달 재테크 준비 원 큐 수 공공 조달 플랫 폼 해당 서비...,주식회사 자영업 공공 조달 재테크 준비 공공 조달 플랫 해당 서비스 공공 조달 물품...,"[주식회사, 자영업, 공공, 조달, 재테크, 준비, 공공, 조달, 플랫, 해당, 서..."
2,2,2021-12-24T16:25:00,아시아경제,https://news.naver.com/main/read.naver?mode=LS...,경제,증권,"인콘, 도중식·최영훈 각자대표 체제로 변경","인콘은 경영 효율성 제고를 위한 조직 개편 차원에서 도중식, 최영훈 각자대표 체제로...",NaN,인 콘 경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 일 공시,인 콘 경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 일 공시,경영 효율 제고 조직 개편 차원 중식 최영훈 각자 대표 체제 변경 공시,"[효율, 제고, 조직, 개편, 차원, 중식, 최영훈, 각자, 체제, 변경, 공시]"
3,3,2021-08-31T10:32:00,데일리안,https://news.naver.com/main/read.naver?mode=LS...,경제,경제 일반,"인콘 조달플랫폼, 조달테크 서비스 통해 N잡 시대 새로운 대안으로 떠올라",최근 MZ세대에서는 본업 외 시간을 활용하는 부업을 떠나서 'N잡러'라는 키워드가 ...,NaN,최근 세대 본업 외 시간 활용 부업 키워드 주목 이 종사 본업 외 파트타임 바 음식...,최근 세대 본업 외 시간 활용 부업 키워드 주목 이 종사 본업 외 파트타임 바 음식...,최근 세대 본업 시간 활용 부업 키워드 주목 종사 본업 파트타임 음식 배달 서비스 ...,"[세대, 본업, 활용, 부업, 키워드, 주목, 종사, 본업, 파트타임, 음식, 배달..."
4,4,2021-09-01T09:45:00,매일신문,https://news.naver.com/main/read.naver?mode=LS...,경제,경제 일반,"인콘, MZ세대 ‘N잡러’를 위한 공공조달플랫폼 서비스 제공",'N잡러'를 위한 공공조달플랫폼 서비스가 많은 관심을 받고 있다. 흔히 말하는 MZ...,NaN,공공 조달 플랫 폼 서비스 관심 말 세대 본업 외 시간 활용 부업 이젠 선택 필수 ...,공공 조달 플랫 폼 서비스 관심 말 세대 본업 외 시간 활용 부업 이젠 선택 필수 ...,공공 조달 플랫 서비스 관심 세대 본업 시간 활용 부업 이젠 선택 필수 추세 파트타...,"[공공, 조달, 플랫, 서비스, 관심, 세대, 본업, 활용, 부업, 이젠, 선택, ..."


###csv 저장
- PRO는 불용화까지 한 내용이다.

In [ ]:
#데이터프레임에서 'regex' 열에 대한 전처리 수행
df['PRO'] = df['PRO'].apply(preprocess)

# 전처리된 데이터프레임을 CSV 파일로 저장
df.to_csv('CHOICHOI.csv', index=False)

print("변환이 완료되었습니다.")

변환이 완료되었습니다.


##토큰화 + 토큰리스트 생성

In [ ]:
def make_tokens (df):
    for i, row in df.iterrows():
        if i%1000==0:
            print(i, '/',len (df))
        token = preprocess(df ['PRO'][i])
        df['PRO'][i] = ' '.join(token)
    return df
df = make_tokens (df)

0 / 11770


<ipython-input-17-0fdc1ad407d4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PRO'][i] = ' '.join(token)


1000 / 11770
2000 / 11770
3000 / 11770
4000 / 11770
5000 / 11770
6000 / 11770
7000 / 11770
8000 / 11770
9000 / 11770
10000 / 11770
11000 / 11770


##Gensim 패키지

In [ ]:
!pip install gensim
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [ ]:
tokenized_docs = df['PRO'].apply(lambda x : x.split())

id2word = corpora. Dictionary (tokenized_docs)
corpus_TDM = [id2word.doc2bow (doc) for doc in tokenized_docs]
tfidf = TfidfModel (corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 50

lda = LdaModel(corpus=corpus_TFIDF,
                id2word=id2word,
                num_topics=n,
                random_state=100)

for t in lda. print_topics():
    print(t)

# df['PRO']

(17, '0.014*""\'대명\',"," + 0.014*""\'리퍼블릭\',"," + 0.012*""\'코콤\',"," + 0.011*""\'코위버\',"," + 0.011*""\'스페이스\',"," + 0.010*""\'에어\',"," + 0.010*""\'에프\',"," + 0.009*""\'데일리\',"," + 0.009*""\'한화\',"," + 0.009*""\'네이버\']"]"')
(7, '0.023*""\'제주도\',"," + 0.009*""\'내정자\',"," + 0.003*""\'박대수\',"," + 0.002*""\'자립\',"," + 0.002*""\'내정\',"," + 0.001*""\'알뜰\',"," + 0.001*""\'이마트\',"," + 0.001*""\'비서\',"," + 0.001*""\'통사\',"," + 0.001*""\'요금제\',","')
(29, '0.004*""\'관중\',"," + 0.004*""\'박람회\',"," + 0.003*""\'야구\',"," + 0.003*""\'스테이트\',"," + 0.002*""\'창원\',"," + 0.001*""\'김해\',"," + 0.001*""\'스티커\',"," + 0.001*""\'이력서\',"," + 0.001*""\'헤럴드\',"," + 0.001*""\'개막\',","')
(9, '0.011*""\'로봇\',"," + 0.006*""\'빌리\',"," + 0.004*"["[\'데일리\',"," + 0.003*""\'데일리\',"," + 0.003*""\'엠게임\',"," + 0.003*""\'넷마블\',"," + 0.003*""\'촉식\',"," + 0.003*""\'이티\',"," + 0.003*""\'비접\',"," + 0.002*""\'주연\',","')
(42, '0.006*""\'반출\',"," + 0.002*""\'반등세\',"," + 0.002*""\'선전\',"," + 0.002*""\'리얼\',"," + 0.002*""\'머티\',"," + 0.

##시각화

In [ ]:
!pip install pyLDAvis==3.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 33.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

In [ ]:
# import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
pip install "pandas<2.0.0"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 46.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.2
    Uninstalling pandas-2.1.2:
      Successfully uninstalled pandas-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [ ]:
tokenized_docs = df['PRO'].apply(lambda x : x.split())

id2word = corpora. Dictionary (tokenized_docs)
corpus_TDM = [id2word.doc2bow (doc) for doc in tokenized_docs]
tfidf = TfidfModel (corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 50

lda = LdaModel(corpus=corpus_TFIDF,
                id2word=id2word,
                num_topics=n,
                random_state=100)

for t in lda. print_topics():
    print(t)

# df['PRO']

# 토픽 모델링 결과를 pyLDAvis 형식으로 변환
lda_display = gensimvis.prepare(lda, corpus_TFIDF, id2word)
# pyLDAvis 결과 시각화
pyLDAvis.display(lda_display)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(17, '0.014*""\'대명\',"," + 0.014*""\'리퍼블릭\',"," + 0.012*""\'코콤\',"," + 0.011*""\'코위버\',"," + 0.011*""\'스페이스\',"," + 0.010*""\'에어\',"," + 0.010*""\'에프\',"," + 0.009*""\'데일리\',"," + 0.009*""\'한화\',"," + 0.009*""\'네이버\']"]"')
(7, '0.023*""\'제주도\',"," + 0.009*""\'내정자\',"," + 0.003*""\'박대수\',"," + 0.002*""\'자립\',"," + 0.002*""\'내정\',"," + 0.001*""\'알뜰\',"," + 0.001*""\'이마트\',"," + 0.001*""\'비서\',"," + 0.001*""\'통사\',"," + 0.001*""\'요금제\',","')
(29, '0.004*""\'관중\',"," + 0.004*""\'박람회\',"," + 0.003*""\'야구\',"," + 0.003*""\'스테이트\',"," + 0.002*""\'창원\',"," + 0.001*""\'김해\',"," + 0.001*""\'스티커\',"," + 0.001*""\'이력서\',"," + 0.001*""\'헤럴드\',"," + 0.001*""\'개막\',","')
(9, '0.011*""\'로봇\',"," + 0.006*""\'빌리\',"," + 0.004*"["[\'데일리\',"," + 0.003*""\'데일리\',"," + 0.003*""\'엠게임\',"," + 0.003*""\'넷마블\',"," + 0.003*""\'촉식\',"," + 0.003*""\'이티\',"," + 0.003*""\'비접\',"," + 0.002*""\'주연\',","')
(42, '0.006*""\'반출\',"," + 0.002*""\'반등세\',"," + 0.002*""\'선전\',"," + 0.002*""\'리얼\',"," + 0.002*""\'머티\',"," + 0.

In [ ]:
# 그래프 이름 : Intertopic Distance Map : 주제간 거리지도
# 단어
# Selected Topic : 토픽지정
# Previous Topic : 이전 주제
# Next Topic : 다음주제
# Clear Topic : 주제 지우기

# Slide to adjust relevance metric :관련성 지표
# Top-30 Most Relevant Terms for Topic X (2.3% of tokens) : 토큰 X에 관련한 가장 연관있는 단어
#

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
